## Update an existing drilling campaign object

This example uses the Evo Python SDK and shows how to download an existing drilling campaign object, apply new data to it, and publish it as a new version of that object.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
import pandas as pd

from evo.notebooks import FeedbackWidget, ServiceManagerWidget

cache_location = "data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
from evo.objects import ObjectAPIClient

# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Define helper functions

These functions assist with assembling the elements and components of geoscience objects and for viewing the new object in the Evo portal.

In [ ]:
import numpy as np


def create_hole_id_mapping(hole_id_table, value_list):
    """
    Create a hole ID mapping table based on the hole ID table and the value list.

    Args:
        hole_id_table (pd.DataFrame): The hole ID lookup table.
        value_list (pd.DataFrame): The value list to create the mapping from.

    Returns:
        mapping_df (pd.DataFrame): The hole ID mapping table.
    """

    num_keys = len(hole_id_table.index)

    mapping_df = pd.DataFrame(list())
    mapping_df["hole_index"] = hole_id_table["key"]
    mapping_df["offset"] = [0] * num_keys
    mapping_df["count"] = [0] * num_keys

    mapping_df["hole_index"] = mapping_df["hole_index"].astype("int32")
    mapping_df["offset"] = mapping_df["offset"].astype("uint64")
    mapping_df["count"] = mapping_df["count"].astype("uint64")

    prev_value = ""
    key = ""
    count = 0
    offset = 0

    for index, row in value_list.iterrows():
        new_value = row["data"]

        if new_value != prev_value:
            if prev_value != "":
                mapping_df.loc[mapping_df["hole_index"] == key, "count"] = count
                mapping_df.loc[mapping_df["hole_index"] == key, "offset"] = offset
                offset += count

            mask = hole_id_table["value"] == new_value
            masked_df = hole_id_table[mask]
            try:
                key_row = masked_df.iloc[[0]]
            except IndexError:
                print("Ignoring this hole ID")
                continue

            key = key_row["key"].iloc[0]
            count = 1
            prev_value = new_value
        else:
            count += 1

    mapping_df.loc[mapping_df["hole_index"] == key, "count"] = count
    mapping_df.loc[mapping_df["hole_index"] == key, "offset"] = offset

    return mapping_df


def create_category_lookup_and_values(attribute):
    """
    Create a category lookup table and the associated column of mapped key values.

    Args:
        attribute (pd.DataFrame): An attribute of a geoscience object.

    Returns:
        table_df (pd.DataFrame): The category lookup table.
        values_df (pd.DataFrame): The associated column with mapped key values.
    """

    # Replace NaN with empty string
    attribute.replace(np.nan, "", regex=True, inplace=True)
    set_obj = set(attribute["data"])
    list_obj = list(set_obj)
    list_obj.sort()
    num_unique_elements = len(list_obj)

    # Create lookup table
    table_df = pd.DataFrame([])
    table_df["key"] = list(range(0, num_unique_elements))
    table_df["value"] = list_obj

    # Create data column
    values_df = pd.DataFrame([])
    values_df["data"] = attribute["data"].map(table_df.set_index("value")["key"])
    return table_df, values_df

### Select the drilling-campaign object to be updated

The example below uses a filter to only show relevant objects in the workspace.

In [ ]:
from prettytable import PrettyTable

all_objects = await object_client.list_all_objects()

table = PrettyTable(["ID", "Name", "Schema"])
for index, obj in enumerate(all_objects):
    if "drilling-campaign" in obj.schema_id.sub_classification:
        table.add_row([str(obj.id).ljust(40), obj.name.ljust(40), obj.schema_id.sub_classification.ljust(40)])

if len(table.rows) == 0:
    print("No drilling campaigns found. Publish a drilling campaign using the 'publish-drilling-campaign' notebook.")
else:
    print(table)

### Download the object files

In [ ]:
object_id = "3824d4a5-586d-4a3c-88e5-14eb6dc9dd9d"  # Replace with your selected object ID

downloaded_object = await object_client.download_object_by_id(object_id=object_id)

metadata = downloaded_object.metadata
downloaded_dict = downloaded_object.as_dict()


def download_table(table_info, fb=None):
    if fb is None:
        fb = FeedbackWidget("Downloading unknown table")
    return data_client.download_table(
        object_id=metadata.id, version_id=metadata.version_id, table_info=table_info, fb=fb
    )


df = pd.DataFrame()

# Use the data client to download the coordinate data.
hole_indices = (
    await download_table(
        downloaded_dict["hole_id"]["values"],
        fb=FeedbackWidget(f"Downloading hole indices data as '{downloaded_dict['hole_id']['values']['data']}'"),
    )
).to_pandas()
index_to_name_map = (
    await download_table(
        downloaded_dict["hole_id"]["table"],
        fb=FeedbackWidget(
            f"Downloading hole index to name map data as '{downloaded_dict['hole_id']['table']['data']}'"
        ),
    )
).to_pandas()
names = pd.DataFrame({"key": hole_indices["data"]}).merge(index_to_name_map, on="key", how="left")["value"]

collar_locations = (
    await download_table(
        downloaded_dict["planned"]["collar"]["coordinates"],
        fb=FeedbackWidget(
            f"Downloading collar locations data as '{downloaded_dict['planned']['collar']['coordinates']['data']}'"
        ),
    )
).to_pandas()
hole_lengths = (
    await download_table(
        downloaded_dict["planned"]["collar"]["distances"],
        fb=FeedbackWidget(
            f"Downloading collar hole distances table as '{downloaded_dict['planned']['collar']['distances']['data']}'"
        ),
    )
).to_pandas()
chunk_data = (
    await download_table(
        downloaded_dict["planned"]["collar"]["holes"],
        fb=FeedbackWidget(
            f"Downloading collar chunks data as '{downloaded_dict['planned']['collar']['holes']['data']}'"
        ),
    )
).to_pandas()

attributes = []
# Use the data client to download the attribute data and merge it with the coordinates data.
for attribute in downloaded_dict["planned"]["collar"]["attributes"]:
    attribute_name = attribute["name"]
    attribute_type = attribute["attribute_type"]

    # Download the attribute data. Every attribute has a 'values' data file.
    values_data = (
        await data_client.download_table(
            object_id=metadata.id,
            version_id=metadata.version_id,
            table_info=attribute["values"],
            fb=FeedbackWidget(
                f"Downloading attribute '{attribute_name}' values data as '{attribute['values']['data']}'"
            ),
        )
    ).to_pandas()
    attributes.append(values_data)

df = pd.concat([names, collar_locations, hole_lengths, chunk_data, *attributes], axis=1)

path_data = (
    await download_table(
        downloaded_dict["planned"]["path"],
        fb=FeedbackWidget(f"Downloading collar path data as '{downloaded_dict['planned']['path']['data']}'"),
    )
).to_pandas()

processed_path_data = pd.concat(
    [
        path_data.iloc[start : start + length].reset_index(drop=True)
        for start, length in zip(chunk_data["offset"], chunk_data["count"])
    ],
    axis=0,
).reset_index(drop=True)
hole_name = []

for hole_id, count in zip(chunk_data["hole_index"], chunk_data["count"]):
    hole_name.extend([index_to_name_map[index_to_name_map["key"] == hole_id]["value"].values[0]] * count)
processed_path_data["hole_name"] = hole_name
processed_path_data = processed_path_data[
    ["hole_name"] + [col for col in processed_path_data.columns if col != "hole_name"]
]

attribute_tables = {}
# Use the data client to download the attribute data and merge it with the coordinates data.
if "collections" in downloaded_dict["planned"]:
    for attribute_table in downloaded_dict["planned"]["collections"]:
        collection_type = attribute_table["collection_type"]
        collection_name = f"planned_{collection_type}_({attribute_table['name']})"

        if collection_type == "interval":
            distance_container = attribute_table["from_to"]["intervals"]["start_and_end"]
            attribute_container = attribute_table["from_to"]["attributes"]
            distance_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=distance_container,
                    fb=FeedbackWidget(f"Downloading distance data as '{distance_container['data']}'"),
                )
            ).to_pandas()
        elif collection_type == "distance":
            distance_container = attribute_table["intervals"]["start_and_end"]
            attribute_container = attribute_table["distance"]["attributes"]
            distance_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=distance_container,
                    fb=FeedbackWidget(f"Downloading distance data as '{distance_container['data']}'"),
                )
            ).to_pandas()
        else:
            continue

        attribute_chunk_data = (
            await data_client.download_table(
                object_id=metadata.id,
                version_id=metadata.version_id,
                table_info=attribute_table["holes"],
                fb=FeedbackWidget(f"Downloading attribute chunk data as '{attribute_table['holes']['data']}'"),
            )
        ).to_pandas()

        columns = [distance_data]
        for column in attribute_container:
            attribute_name = column["name"]
            attribute_type = column["attribute_type"]

            column_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=column["values"],
                    fb=FeedbackWidget(
                        f"Downloading attribute '{attribute_name}' column data as '{column['values']['data']}'"
                    ),
                )
            ).to_pandas()
            column_data.columns = ["data"]

            # If the attribute is a category, download the 'table' data as well.
            if attribute_type == "category":
                lookup_table = (
                    await data_client.download_table(
                        object_id=metadata.id,
                        version_id=metadata.version_id,
                        table_info=column["table"],
                        fb=FeedbackWidget(
                            f"Downloading attribute '{attribute_name}' lookup table data as '{column['table']['data']}'"
                        ),
                    )
                ).to_pandas()

                # Merge the values data with the table data.
                merged_data = pd.merge(column_data, lookup_table, left_on="data", right_on="key", how="left")
                # Drop the 'data' and 'key' columns from the merged data.
                merged_data.drop(columns=["data", "key"], inplace=True)
                # Rename the 'value' column to the attribute name.
                merged_data.rename(columns={"value": attribute_name}, inplace=True)
                # Concatenate the merged data with the coordinates data.
                columns.append(merged_data)

            elif attribute_type == "scalar":
                # Rename the 'data' column to the attribute name.
                column_data.rename(columns={"data": attribute_name}, inplace=True)
                # Concatenate the data with the coordinates data.
                columns.append(column_data)

            else:
                raise ValueError(f"Unknown attribute type: {attribute_type}")

        attribute_tables[collection_name] = pd.concat(columns, axis=1)
        attribute_tables[f"{collection_name} (Cnk)"] = attribute_chunk_data